In [28]:
import pandas as pd
import numpy as np
import os
import sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

import pyperclip as clip

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:20,.2f}'.format

In [29]:
cd

/Users/karatechop


In [30]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)/Wave 1/LAPD

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)/Wave 1/LAPD


In [31]:
ls

ALL-LAPD-AB953.csv   LAPD-AB953-2019.csv  LAPD-AB953-2021.csv
LAPD-AB953-2018.csv  LAPD-AB953-2020.csv  dictionary.xlsx


In [32]:
warnings.filterwarnings("ignore") 

la18 = pd.read_csv('LAPD-AB953-2018.csv')
la19 = pd.read_csv('LAPD-AB953-2019.csv')
la20 = pd.read_csv('LAPD-AB953-2020.csv')
la21 = pd.read_csv('LAPD-AB953-2021.csv')

la_dfs = [la18,la19,la20,la21]

In [37]:
la18.columns

Index(['idx', 'Agency', 'Seq', 'txType', 'NFIA', 'StopDate', 'StopTime',
       'Duration', 'SvcCall', 'Officer', 'Experience', 'AsgntType',
       'Assignment', 'Proxy', 'Addr', 'City', 'K12', 'SchoolCode', 'pid',
       'race', 'sex', 'nonConf', 'lgbt', 'age', 'noEngl', 'disab', 'student',
       'reason', 'rsnDsc', 'tpMvViol', 'mvCJIS', 'eduCode', 'eduSub',
       'suspCJIS', 'suspBase', 'act', 'srchBase', 'srchDsc', 'szrBase',
       'propSzd', 'evid', 'res', 'warnCJIS', 'citCJIS', 'infCJIS', 'arrCJIS'],
      dtype='object')

In [42]:
la18['Officer'].value_counts()

0    336681
Name: Officer, dtype: int64

In [33]:
def convert_simplify(df):
    i = df
    i = i.rename(columns={'StopDate':'Date',
                          'StopTime':'time',
                          'Duration':'Duration',
                          'Experience':'Experience',
                          'pid':'PID',
                          'race':'race',
                          'sex':'sex',
                          'age':'age',
                          'tpMvViol':'type',
                          'reason':'reason',
                          'suspBase':'basis',
                          'act':'actions taken',
                          'srchBase':'search basis',
                          'szrBase':'seizure basis',
                          'propSzd':'property seized',
                          'evid':'evidence',
                          'res':'result',
                          'Addr':'location'})

    i = i[(i['SvcCall']!='Y')\
#           & ((i['type']==1) | (i['type']==2))\
          &  (i['AsgntType']==1)\
          &  (i['K12']!='Y')\
          &  (i['student']!='Y')]

    i.loc[i['type'] == 1,'type'] = '1 Moving'
    i.loc[i['type'] == 2,'type'] = '2 Equipment'
    i.loc[i['type'] == 3,'type'] = '3 Non-Moving'
    
    try:     
        i.loc[i['race'] == '1','race'] = 'asian'
        i.loc[i['race'] == '2','race'] = 'black/african american'
        i.loc[i['race'] == '3','race'] = 'hispanic/latino(a)'
        i.loc[i['race'] == '4','race'] = 'middle eastern or south asian'
        i.loc[i['race'] == '5','race'] = 'native american'
        i.loc[i['race'] == '6','race'] = 'pacific islander'
        i.loc[i['race'] == '7','race'] = 'white'
        i.loc[i['race'] == 1,'race'] = 'asian'
        i.loc[i['race'] == 2,'race'] = 'black/african american'
        i.loc[i['race'] == 3,'race'] = 'hispanic/latino(a)'
        i.loc[i['race'] == 4,'race'] = 'middle eastern or south asian'
        i.loc[i['race'] == 5,'race'] = 'native american'
        i.loc[i['race'] == 6,'race'] = 'pacific islander'
        i.loc[i['race'] == 7,'race'] = 'white'
    except: 
        print('race switch failed')
        pass
    
    try: #mixed list type 1
        mixed_list = []
        for check in i['race']:
            if len(check.split(','))>1:
                mixed_list.append('mixed')
            else: 
                mixed_list.append(check)
        i['race'] = mixed_list
        i['race'] = i['race'].str.lower()
    except:
        try: #mixed list type 2
            mixed_list = []
            for check in i['race']:
                if len(check.split('or'))>1:
                    mixed_list.append(check.split('or')[0].replace(' ',''))
                else: 
                    mixed_list.append(check)
            i['race'] = mixed_list
            i['race'] = i['race'].str.lower()
        except:    
            print('no mixed race changes')
            pass



    
    try: 
        i = i.drop_duplicates(subset=['stop_id'], keep='first')
    except: 
        print('no dropped pid dupes')
        pass

    output_df = i[['Date', 'time','Duration','Experience','PID','race', 'sex','age','type',
                   'reason','basis', 'actions taken', 'search basis', 'seizure basis','property seized', 
                   'evidence', 'result','location']]
    
    
    return output_df

In [34]:
year = 2018
for i in la_dfs:  
    if 'AB953' not in i:
        df = convert_simplify(i)

        if year == 2018:
            LAPD = convert_simplify(la18)
            
        else:
            LAPD = pd.merge(LAPD,df,how='outer')
            
        print(f'formatted la{year} df is {df.shape}')
        print(f'la{year} is merged! LAPD shape is now {LAPD.shape}')
        year+=1
LAPD.columns = LAPD.columns.str.lower()
LAPD.to_csv('ALL-LAPD-AB953.csv',index=False)

no dropped pid dupes
no dropped pid dupes
formatted la2018 df is (275155, 18)
la2018 is merged! LAPD shape is now (275155, 18)
no dropped pid dupes
formatted la2019 df is (607208, 18)
la2019 is merged! LAPD shape is now (882363, 18)
no dropped pid dupes
formatted la2020 df is (400242, 18)
la2020 is merged! LAPD shape is now (1282605, 18)
no dropped pid dupes
formatted la2021 df is (320372, 18)
la2021 is merged! LAPD shape is now (1602977, 18)


In [36]:
la18['experience'].value_counts()

KeyError: 'experience'

In [13]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [14]:
LAPD.loc[LAPD['type']=='middle eastern or south asian','type']=None
LAPD.loc[LAPD['type']=='middle eastern or south asian','type']=None
LAPD.loc[LAPD['duration']=='0-9','duration']=5
 

type_list=[]
LAPD['type'] = LAPD['type'].astype(str)

for i in LAPD['type']:
    if '1' in i:
        type_list.append('Moving')
    elif '2' in i:
        type_list.append('Equipment')
    elif '3' in i:
        type_list.append('Non-Moving')
    
    elif 'equipment' in i.lower():
        type_list.append('Equipment')
    elif 'non' in i.lower():
        type_list.append('Non-Moving')
    else:
        type_list.append('Moving')
LAPD['type'] = type_list


LAPD.loc[LAPD['race'] == 'Asian','race'] = 'asian'    
LAPD.loc[LAPD['race'] == 'Black/African American','race'] = 'black/african american'    
LAPD.loc[LAPD['race'] == 'black/african american','race'] = 'black/african american'    

LAPD.loc[LAPD['race'] == 'Hispanic/Latino','race'] = 'hispanic/latino(a)'   
LAPD.loc[LAPD['race'] == 'Hispanic/Latino/a','race'] = 'hispanic/latino(a)'    
LAPD.loc[LAPD['race'] == 'White','race'] = 'white'
LAPD.loc[LAPD['race'] == 'asian','race'] = 'asian'    
LAPD.loc[LAPD['race'] == 'black/african american','race'] = 'black/african american'    
LAPD.loc[LAPD['race'] == 'hispanic/latino','race'] = 'hispanic/latino(a)'    

LAPD.loc[LAPD['race'] == 'white','race'] = 'white'

LAPD.loc[LAPD['race'] == '1','race'] = 'asian'
LAPD.loc[LAPD['race'] == '2','race'] = 'black/african american'
LAPD.loc[LAPD['race'] == '3','race'] = 'hispanic/latino(a)'
LAPD.loc[LAPD['race'] == 'hispanic/latino/a','race'] = 'hispanic/latino(a)'
LAPD.loc[LAPD['race'] == '7','race'] = 'white'
LAPD.loc[LAPD['race'] == 1,'race'] = 'asian'
LAPD.loc[LAPD['race'] == 2,'race'] = 'black/african american'
LAPD.loc[LAPD['race'] == 3,'race'] = 'hispanic/latino(a)'
LAPD.loc[LAPD['race'] == 7,'race'] = 'white'

LAPD['race'] = LAPD['race'].astype(str)
mixed_list = []
for i in LAPD['race']:
    if i not in ['white','asian','black/african american','hispanic/latino(a)']:
        mixed_list.append('group 4')
    else:
        mixed_list.append(i)
LAPD['race'] = mixed_list
      


sex_list = []
LAPD['sex'] = LAPD['sex'].astype(str)

LAPD.loc[LAPD['sex'] == 1,'sex'] = 'male'
LAPD.loc[LAPD['sex'] == 'Male','sex'] = 'male'
LAPD.loc[LAPD['sex'] == 2,'sex'] = 'female'
LAPD.loc[LAPD['sex'] == 'Female','sex'] = 'female'
LAPD.loc[LAPD['sex']==3,'sex'] = None
LAPD.loc[LAPD['sex']==4,'sex'] = None

def sex_code(i):
    if '1' in i:
        return 'Male'
    elif '2' in i:
        return 'Female'
    elif 'female' in i or 'Female' in i:
        return 'Female'
    elif 'male' in i or 'Male' in i:
        return 'Male'
    else:
        return None
        
LAPD['sex'] = [sex_code(i) if i is not None else i for i in LAPD['sex']]
# [f(x) if x is not None else '' for x in xs]        
LAPD['sex'].value_counts()


KeyError: 'duration'